In [92]:
import os

In [93]:
%pwd

'C:\\datascience\\Projects\\text_summarizer'

In [94]:
import os
os.chdir(r"C:\datascience\Projects\text_summarizer")


In [95]:
%pwd

'C:\\datascience\\Projects\\text_summarizer'

In [96]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [97]:
from textSummarizer.constants import *
from textSummarizer.utils.common import create_directory,read_yaml_config,get_size
from textSummarizer.logger_config.config import logger
import urllib.request as request
import zipfile


In [98]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml_config(config_filepath)
        self.params = read_yaml_config(params_filepath)

        create_directory([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directory([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [99]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [100]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-04-05 20:15:20,123: INFO: Successfully loaded configuration from: config\config.yaml]
[2025-04-05 20:15:20,126: INFO: Successfully loaded configuration from: params.yaml]
[2025-04-05 20:15:20,129: INFO: Created directory at: artifacts]


[2025-04-05 20:15:20,131: INFO: Created directory at: artifacts/data_ingestion]
[2025-04-05 20:15:24,596: INFO: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 86E0:1AC85B:742EA:11679A:67F1405D
Accept-Ranges: bytes
Date: Sat, 05 Apr 2025 14:44:58 GMT
Via: 1.1 varnish
X-Served-By: cache-del21736-DEL
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1743864298.479613,VS0,VE250
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: f75a73c169a4e2cc38b56f3c29d8b8e303aa0c90
Expires: Sat, 05 Apr 2025 14:49:5

In [101]:
from textSummarizer.utils.common import read_yaml_config
from pathlib import Path

cfg = read_yaml_config(Path("config/config.yaml"))
print(cfg)


[2025-04-05 20:15:24,797: INFO: Successfully loaded configuration from: config\config.yaml]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}}
